libraries

In [1]:
from langgraph.graph import END, START, StateGraph
from langchain_ollama import ChatOllama
from typing import TypedDict , Annotated , Literal
from pydantic import BaseModel,Field

model local llm

In [2]:
# -----------------------
# LLM SETUP
# -----------------------
model = ChatOllama(
    model="llama3.2:3b",
    temperature=0.7
)

1. sentiment extraction

In [3]:
# -----------------------
# SENTIMENT SCHEMA
# -----------------------
class SentimentSchema(BaseModel):
    sentiment: Literal["positive", "negative"]


structured_model = model.with_structured_output(SentimentSchema)


In [4]:
prompt = f"""
what is the sentiment of this review - The software is good
"""
structured_model.invoke(prompt).sentiment

'positive'

define state

In [5]:
# -----------------------
# STATE
# -----------------------
class ReviewState(TypedDict):
    review: str
    sentiment: Literal["positive", "negative"]
    diagnosis: dict
    response: str


functions

In [6]:
def find_sentiment(state: ReviewState) -> ReviewState:
    prompt = f"""
    For the following review, find the sentiment:

    "{state['review']}"
    """

    result = structured_model.invoke(prompt).sentiment

    return {**state, "sentiment": result}

In [7]:
def check_sentiment(state: ReviewState) -> Literal["positive_response", "run_diagnosis"]:
    if state["sentiment"] == "positive":
        return "positive_response"
    else:
        return "run_diagnosis"

In [8]:
def positive_response(state: ReviewState) -> ReviewState:
    prompt = f"""
    Write a warm, friendly thank-you message in response to this review:

    "{state['review']}"

    Also ask the user to leave feedback on our website.
    """

    reply = model.invoke(prompt).content

    return {**state, "response": reply}

schema for structured o/p for negative response

In [9]:
class DiagnosisSchema(BaseModel):
    issue_type: Literal["product", "service", "pricing", "delivery"] = Field(description="The type of issue")
    tone: Literal["positive", "negative", "neutral","calm", "angry"] = Field(description="The tone expressed by user for the review")
    urgency: Literal["low", "medium", "high"] = Field(description="The urgency of the issue")

In [10]:
structured_model2 = model.with_structured_output(DiagnosisSchema)

In [11]:
def run_diagnosis(state: ReviewState) -> ReviewState:

    prompt = f"""
    Diagnose this negative review:

    "{state["review"]}"

    Return issue_type, tone, and urgency.
    """

    response = structured_model2.invoke(prompt)

    return {
        **state,
        "diagnosis": response.model_dump()   
    }


In [ ]:
def negative_response(state: ReviewState) -> ReviewState:
    diagnosis = state["diagnosis"]

    prompt = f"""
    You are a support assistant.

    The user had a "{diagnosis["issue_type"]}" issue
    with tone "{diagnosis["tone"]}" 
    and urgency "{diagnosis["urgency"]}".

    Write an empathetic response and provide a helpful resolution.
    """

    output = model.invoke(prompt).content

    return {
        **state,
        "response": output
    }

graph

In [13]:
graph = StateGraph(ReviewState)

graph.add_node("find_sentiment", find_sentiment)
graph.add_node("positive_response", positive_response)
graph.add_node("run_diagnosis", run_diagnosis)
graph.add_node("negative_response", negative_response)

graph.add_edge(START, "find_sentiment")

graph.add_conditional_edges(
    "find_sentiment",
    check_sentiment,  # returns "positive_response" or "run_diagnosis"
)

graph.add_edge("positive_response", END)
graph.add_edge("run_diagnosis", "negative_response")
graph.add_edge("negative_response", END)

workflow = graph.compile()


In [16]:
initial_state = {
    "review": "The product was delayed and support did not respond."
}


In [15]:
initial_state = {
    "review": "The product is good."
}

In [17]:
final_state = workflow.invoke(initial_state)

In [18]:
print(final_state)

{'review': 'The product was delayed and support did not respond.', 'sentiment': 'negative', 'diagnosis': {'issue_type': 'product', 'tone': 'negative', 'urgency': 'low'}, 'response': "I'm so sorry to hear that you're experiencing issues with your product. I can see that you're frustrated, and I want to assure you that I'm here to help.\n\nCan you please tell me more about the issue you're facing? What's not working as expected, and how is it affecting you? I'll do my best to understand the problem and provide a solution.\n\nIn the meantime, I'd like to offer some temporary assistance. Would you be open to receiving a replacement part or a refund, depending on your preference? We want to get this resolved for you as soon as possible.\n\nPlease know that we value your business and appreciate your feedback. Your concerns are important to us, and we'll do our best to make things right.\n\nIf there's anything else I can do to help, please let me know. I'm here to support you, and I want to e